In [2]:
import pandas as pd
import numpy as np
import time
import os
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

In [2]:
os.environ['TORCH_HOME'] = 'D:/torch_cache'  # For pretrained models
os.environ['HF_HOME'] = 'D:/huggingface_cache'  # If using Hugging Face

In [3]:
load_dotenv()

# Use the variables
data_dir = os.getenv("DATA_DIR")
ground_truth_path = os.getenv("GROUND_TRUTH_PATH")
checkpoint_path = os.getenv("CHECKPOINT_BEST_PATH")
checkpoint_dir = os.getenv("CHECKPOINT_DIR")

In [3]:
# PyTorch modules

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import TensorDataset, DataLoader, random_split, Subset, Dataset
import torch.nn.functional as F
import torchvision.models as models
from torchvision.datasets import ImageFolder

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

#### Data loading and augmentation steps

In [5]:
train_transform = transforms.Compose([
    # Spatial augmentations (scenes tolerate more variation than cars)
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),  # Slightly more rotation allowed
    
    # Color augmentations (emotions are sensitive to color/tone)
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
    
    # Additional augmentations for small dataset
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Small shifts
    transforms.RandomPerspective(distortion_scale=0.2, p=0.3),
    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Conforms to ImageNet normalization 
    
    # Regularization (critical for ~2K images)
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.15))
])


In [6]:
eval_transform = transforms.Compose([
    transforms.Resize(256),  # Resize shorter side to 256
    transforms.CenterCrop(224),  # Standard crop
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [7]:
# Load dataset WITHOUT transforms (for indexing only)
data_dir = r"training/dataset/images"

In [8]:
# Step 1: Load dataset without transform to get targets for stratification
full_dataset = ImageFolder(data_dir, transform=None)
targets = np.array(full_dataset.targets)

print(f"Total samples: {len(full_dataset)}")
print(f"Class names: {full_dataset.classes}")

Total samples: 1980
Class names: ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']


In [9]:
# Step 2: First split - 70% train, 30% temp (val + test)
train_idx, temp_idx = train_test_split(
    np.arange(len(targets)),
    test_size=0.30,           # 30% for val + test combined
    stratify=targets,         # Ensures equal class proportions
    random_state=42           # For reproducibility
)

In [10]:
# Step 3: Second split - divide temp into 15% val and 15% test
val_idx, test_idx = train_test_split(
    temp_idx,
    test_size=0.5,            # 50% of 30% = 15% of total
    stratify=targets[temp_idx],  # Maintain stratification
    random_state=42
)

In [11]:
# Step 4: Create Subsets
train_subset = Subset(full_dataset, train_idx)
val_subset = Subset(full_dataset, val_idx)
test_subset = Subset(full_dataset, test_idx)

In [12]:
# Step 5: Wrapper to apply different transforms to different subsets
class TransformDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform
        
    def __len__(self):
        return len(self.subset)
    
    def __getitem__(self, idx):
        # Get PIL Image and label from subset
        image, label = self.subset[idx]
        
        # Apply transform if specified
        if self.transform:
            image = self.transform(image)
            
        return image, label

In [13]:
# Apply transforms
train_dataset = TransformDataset(train_subset, train_transform)
val_dataset = TransformDataset(val_subset, eval_transform)
test_dataset = TransformDataset(test_subset, eval_transform)

In [14]:
# Step 6: Create DataLoaders
batch_size = 32
num_workers = 0  # Use 0 on Windows to avoid multiprocessing issues; use 4 on Linux

In [15]:
train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True,           # Only training set gets shuffled
    num_workers=num_workers
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False,          # No shuffle for validation
    num_workers=num_workers
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    shuffle=False,          # No shuffle for test
    num_workers=num_workers
)

In [16]:
# Verification: Check class distribution
def check_class_distribution(subset, targets, name):
    subset_targets = targets[subset.indices]
    unique, counts = np.unique(subset_targets, return_counts=True)
    print(f"\n{name} set ({len(subset)} samples):")
    for cls_idx, count in zip(unique, counts):
        print(f"  {full_dataset.classes[cls_idx]}: {count}")

In [17]:
check_class_distribution(train_subset, targets, "Train")
check_class_distribution(val_subset, targets, "Validation")
check_class_distribution(test_subset, targets, "Test")


Train set (1386 samples):
  anger: 231
  disgust: 231
  fear: 231
  joy: 231
  sadness: 231
  surprise: 231

Validation set (297 samples):
  anger: 50
  disgust: 50
  fear: 49
  joy: 49
  sadness: 49
  surprise: 50

Test set (297 samples):
  anger: 49
  disgust: 49
  fear: 50
  joy: 50
  sadness: 50
  surprise: 49


#### Training & validation function

In [18]:
def train_and_validate(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    epochs,
    num_classes=6,
    checkpoint_dir=checkpoint_dir,
    class_names=None
):
    """
    Trains and validates the model for multi-class classification.
    Saves checkpoint after each epoch.
    """

    assert epochs % 1 == 0, "Epochs must be in steps of 1"
    
    # Create checkpoint directory
    os.makedirs(checkpoint_dir, exist_ok=True)

    train_losses = []
    val_losses = []
    val_accuracies = []

    best_val_acc = 0.0  # Track best model

    for epoch in range(epochs):

        # --------------------
        # Training phase
        # --------------------
        model.train()
        running_train_loss = 0.0

        for images, labels in train_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        avg_train_loss = running_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # --------------------
        # Validation phase
        # --------------------
        model.eval()
        running_val_loss = 0.0
        total = 0
        correct = 0

        all_true_labels = []
        all_pred_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(images)
                loss = criterion(outputs, labels)

                running_val_loss += loss.item()

                _, predicted = torch.max(outputs, dim=1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_true_labels.extend(labels.cpu().numpy())
                all_pred_labels.extend(predicted.cpu().numpy())

        avg_val_loss = running_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        accuracy = 100.0 * correct / total
        val_accuracies.append(accuracy)

        # --------------------
        # Logging
        # --------------------
        print(
            f"Epoch [{epoch+1}/{epochs}] | "
            f"Train Loss: {avg_train_loss:.4f} | "
            f"Val Loss: {avg_val_loss:.4f} | "
            f"Val Acc: {accuracy:.2f}%"
        )

        # --------------------
        # Save checkpoint (every epoch)
        # --------------------
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch+1}.pth')
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'val_acc': accuracy,
        }, checkpoint_path)
        
        # --------------------
        # Save best model separately
        # --------------------
        if accuracy > best_val_acc:
            best_val_acc = accuracy
            best_path = os.path.join(checkpoint_dir, 'best_model.pth')
            torch.save(model.state_dict(), best_path)
            print(f"  **** New best model saved! (Acc: {accuracy:.2f}%)")

    # --------------------
    # Final evaluation metrics (after all epochs)
    # --------------------
    print(f"\n{'='*50}")
    print(f"Training completed! Best validation accuracy: {best_val_acc:.2f}%")
    print(f"{'='*50}")
    
    # Classification report for multi-class
    if class_names:
        print("\nClassification Report (Last Epoch):")
        print(classification_report(all_true_labels, all_pred_labels, target_names=class_names))
    
    # Confusion matrix
    cm = confusion_matrix(all_true_labels, all_pred_labels)
    print(f"\nConfusion Matrix:\n{cm}")

    return train_losses, val_losses, val_accuracies, all_true_labels, all_pred_labels


In [19]:
# Load pretrained weights with explicit cache location
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Freeze all parameters first
for param in model.parameters():
    param.requires_grad = False

# Replace fully connected layer with 6 outputs
# --------------------------------------------------
# Get input features of original fc layer (typically 2048 for ResNet50)
in_features = model.fc.in_features

# Replace with new fc layer (unfrozen by default)
model.fc = nn.Linear(in_features, 6)

# Move ENTIRE model to device AFTER architecture changes
model = model.to(device)

In [20]:
# Verification: Check which parameters are trainable

print("Trainable parameters:")
total_params = 0
trainable_params = 0

for name, param in model.named_parameters():
    total_params += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
        print(f"  {name}: {param.shape}")

print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Frozen parameters: {total_params - trainable_params:,}")
print(f"Percentage trainable: {100 * trainable_params / total_params:.2f}%")

Trainable parameters:
  fc.weight: torch.Size([6, 2048])
  fc.bias: torch.Size([6])

Total parameters: 23,520,326
Trainable parameters: 12,294
Frozen parameters: 23,508,032
Percentage trainable: 0.05%


In [21]:
optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

# Class names
emotion_classes = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

In [22]:
# Train
train_losses, val_losses, val_accs, true_labels, pred_labels = train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    epochs=30,
    num_classes=6,
    checkpoint_dir=checkpoint_dir,
    class_names=emotion_classes
)

Epoch [1/30] | Train Loss: 1.5672 | Val Loss: 1.3876 | Val Acc: 46.80%
  **** New best model saved! (Acc: 46.80%)
Epoch [2/30] | Train Loss: 1.2862 | Val Loss: 1.2897 | Val Acc: 49.83%
  **** New best model saved! (Acc: 49.83%)
Epoch [3/30] | Train Loss: 1.1664 | Val Loss: 1.2311 | Val Acc: 51.52%
  **** New best model saved! (Acc: 51.52%)
Epoch [4/30] | Train Loss: 1.1209 | Val Loss: 1.2050 | Val Acc: 53.20%
  **** New best model saved! (Acc: 53.20%)
Epoch [5/30] | Train Loss: 1.0769 | Val Loss: 1.1806 | Val Acc: 52.53%
Epoch [6/30] | Train Loss: 1.0442 | Val Loss: 1.1653 | Val Acc: 54.55%
  **** New best model saved! (Acc: 54.55%)
Epoch [7/30] | Train Loss: 0.9954 | Val Loss: 1.1603 | Val Acc: 51.85%
Epoch [8/30] | Train Loss: 0.9567 | Val Loss: 1.1680 | Val Acc: 53.20%
Epoch [9/30] | Train Loss: 0.9150 | Val Loss: 1.1539 | Val Acc: 54.21%
Epoch [10/30] | Train Loss: 0.9035 | Val Loss: 1.1553 | Val Acc: 54.21%
Epoch [11/30] | Train Loss: 0.8866 | Val Loss: 1.1429 | Val Acc: 53.20%
Ep

In [23]:
# Phase 1: Train only FC (already done - 30 epochs, 57.58%)

# Phase 2: Unfreeze layer4 + train FC (30 epochs)
for param in model.layer4.parameters():
    param.requires_grad = True
    
# Use differential learning rates: lower for pretrained layers
optimizer = torch.optim.Adam([
    {'params': model.layer4.parameters(), 'lr': 1e-5}, # Deeper layers: slower
    {'params': model.fc.parameters(), 'lr': 1e-4} # New layer: faster
])

In [24]:
# 3. Continue training 30 epochs
train_losses, val_losses, val_accs, true_labels, pred_labels = train_and_validate(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    epochs=30,  
    checkpoint_dir=checkpoint_dir,
    class_names=emotion_classes
)

Epoch [1/30] | Train Loss: 0.6409 | Val Loss: 1.1670 | Val Acc: 54.55%
  **** New best model saved! (Acc: 54.55%)
Epoch [2/30] | Train Loss: 0.6391 | Val Loss: 1.1732 | Val Acc: 56.23%
  **** New best model saved! (Acc: 56.23%)
Epoch [3/30] | Train Loss: 0.6023 | Val Loss: 1.1780 | Val Acc: 55.22%
Epoch [4/30] | Train Loss: 0.6006 | Val Loss: 1.1667 | Val Acc: 57.58%
  **** New best model saved! (Acc: 57.58%)
Epoch [5/30] | Train Loss: 0.5658 | Val Loss: 1.1795 | Val Acc: 57.24%
Epoch [6/30] | Train Loss: 0.5553 | Val Loss: 1.2019 | Val Acc: 56.57%
Epoch [7/30] | Train Loss: 0.5034 | Val Loss: 1.1931 | Val Acc: 56.90%
Epoch [8/30] | Train Loss: 0.5308 | Val Loss: 1.2111 | Val Acc: 57.58%
Epoch [9/30] | Train Loss: 0.5106 | Val Loss: 1.2109 | Val Acc: 57.58%
Epoch [10/30] | Train Loss: 0.4761 | Val Loss: 1.2215 | Val Acc: 57.24%
Epoch [11/30] | Train Loss: 0.4676 | Val Loss: 1.2181 | Val Acc: 58.59%
  **** New best model saved! (Acc: 58.59%)
Epoch [12/30] | Train Loss: 0.4549 | Val Loss

#### Modified training loop with LR scheduler

In [37]:
def train_and_validate_LR(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    epochs,
    scheduler=None,           # NEW: Add scheduler parameter
    num_classes=6,
    checkpoint_dir=checkpoint_dir,
    class_names=None
):
    """
    Trains and validates the model with scheduler support.
    """
    
    os.makedirs(checkpoint_dir, exist_ok=True)

    train_losses = []
    val_losses = []
    val_accuracies = []
    best_val_acc = 0.0

    for epoch in range(epochs):

        # --------------------
        # Training phase (same as before)
        # --------------------
        model.train()
        running_train_loss = 0.0

        for images, labels in train_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        avg_train_loss = running_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # --------------------
        # Validation phase (same as before)
        # --------------------
        model.eval()
        running_val_loss = 0.0
        total = 0
        correct = 0
        all_true_labels = []
        all_pred_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images = images.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)

                outputs = model(images)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item()

                _, predicted = torch.max(outputs, dim=1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_true_labels.extend(labels.cpu().numpy())
                all_pred_labels.extend(predicted.cpu().numpy())

        avg_val_loss = running_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        
        accuracy = 100.0 * correct / total
        val_accuracies.append(accuracy)

        # --------------------
        # Logging (same)
        # --------------------
        print(f"Epoch [{epoch+1}/{epochs}] | "
              f"Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | "
              f"Val Acc: {accuracy:.2f}%")

        # --------------------------------------------------
        # NEW: Step the scheduler based on validation accuracy
        # --------------------------------------------------
        if scheduler is not None:
            scheduler.step(accuracy)  # Pass current validation accuracy

        # --------------------
        # Save checkpoint (same)
        # --------------------
        checkpoint_path = os.path.join(checkpoint_dir, f'model_epoch_{epoch+1}.pth')
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': avg_train_loss,
            'val_loss': avg_val_loss,
            'val_acc': accuracy,
        }, checkpoint_path)
        
        # Save best model
        if accuracy > best_val_acc:
            best_val_acc = accuracy
            best_path = os.path.join(checkpoint_dir, 'best_model.pth')
            torch.save(model.state_dict(), best_path)
            print(f"  **** New best model saved! (Acc: {accuracy:.2f}%)")

    print(f"\n{'='*50}")
    print(f"Training completed! Best validation accuracy: {best_val_acc:.2f}%")
    print(f"{'='*50}")

    return train_losses, val_losses, val_accuracies, all_true_labels, all_pred_labels


#### Use regularization & dropout in FC layer, unfreeze convolutions layers 3 and 4

In [38]:
# Load pretrained ResNet50
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# --------------------------------------------------
# Step 1: Replace FC with improved architecture (dropout + deeper layers)
# --------------------------------------------------
in_features = model.fc.in_features

model.fc = nn.Sequential(
    nn.Dropout(0.5),              # Prevent overfitting
    nn.Linear(in_features, 512),  # Intermediate layer
    nn.ReLU(),                    # Non-linearity
    nn.Dropout(0.3),              # Additional regularization
    nn.Linear(512, 6)             # 6 emotion classes
)

# --------------------------------------------------
# Step 2: Unfreeze layer3 AND layer4 for fine-tuning
# --------------------------------------------------
# First freeze everything
for param in model.parameters():
    param.requires_grad = False

# Unfreeze layer3 and layer4 (high-level features)
for param in model.layer3.parameters():
    param.requires_grad = True
for param in model.layer4.parameters():
    param.requires_grad = True

# FC layer is already trainable (new parameters have requires_grad=True by default)

# --------------------------------------------------
# Step 3: Move to device AFTER all modifications
# --------------------------------------------------
model = model.to(device)

# Verify trainable parameters
print("Trainable parameter groups:")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"  {name}: {param.shape}")

Trainable parameter groups:
  layer3.0.conv1.weight: torch.Size([256, 512, 1, 1])
  layer3.0.bn1.weight: torch.Size([256])
  layer3.0.bn1.bias: torch.Size([256])
  layer3.0.conv2.weight: torch.Size([256, 256, 3, 3])
  layer3.0.bn2.weight: torch.Size([256])
  layer3.0.bn2.bias: torch.Size([256])
  layer3.0.conv3.weight: torch.Size([1024, 256, 1, 1])
  layer3.0.bn3.weight: torch.Size([1024])
  layer3.0.bn3.bias: torch.Size([1024])
  layer3.0.downsample.0.weight: torch.Size([1024, 512, 1, 1])
  layer3.0.downsample.1.weight: torch.Size([1024])
  layer3.0.downsample.1.bias: torch.Size([1024])
  layer3.1.conv1.weight: torch.Size([256, 1024, 1, 1])
  layer3.1.bn1.weight: torch.Size([256])
  layer3.1.bn1.bias: torch.Size([256])
  layer3.1.conv2.weight: torch.Size([256, 256, 3, 3])
  layer3.1.bn2.weight: torch.Size([256])
  layer3.1.bn2.bias: torch.Size([256])
  layer3.1.conv3.weight: torch.Size([1024, 256, 1, 1])
  layer3.1.bn3.weight: torch.Size([1024])
  layer3.1.bn3.bias: torch.Size([1024])

#### Optimizer & Scheduler

In [41]:
# --------------------------------------------------
# Differential learning rates
# --------------------------------------------------
optimizer = torch.optim.Adam([
    {'params': model.layer3.parameters(), 'lr': 1e-6},   # Deepest conv: slowest
    {'params': model.layer4.parameters(), 'lr': 1e-5},   # Deep conv: slow
    {'params': model.fc.parameters(), 'lr': 1e-4}        # New layers: faster
])

# --------------------------------------------------
# Learning rate scheduler - ReduceLROnPlateau
# --------------------------------------------------
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='max',           # Monitor validation accuracy (maximize)
    factor=0.5,           # Reduce LR by half when plateau
    patience=5,           # Wait 5 epochs before reducing
    min_lr=1e-7           # Don't go below this
)

criterion = nn.CrossEntropyLoss()

In [44]:
# Train for 25 epochs with scheduler
train_losses, val_losses, val_accs, true_labels, pred_labels = train_and_validate_LR(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    epochs=25,                          # Increased to 25
    scheduler=scheduler,                # Pass the scheduler
    num_classes=6,
    checkpoint_dir=checkpoint_dir,
    class_names=['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
)

Epoch [1/25] | Train Loss: 1.7943 | Val Loss: 1.7869 | Val Acc: 20.88%
  **** New best model saved! (Acc: 20.88%)
Epoch [2/25] | Train Loss: 1.7918 | Val Loss: 1.7845 | Val Acc: 22.22%
  **** New best model saved! (Acc: 22.22%)
Epoch [3/25] | Train Loss: 1.7895 | Val Loss: 1.7826 | Val Acc: 22.22%
Epoch [4/25] | Train Loss: 1.7835 | Val Loss: 1.7774 | Val Acc: 23.91%
  **** New best model saved! (Acc: 23.91%)
Epoch [5/25] | Train Loss: 1.7820 | Val Loss: 1.7733 | Val Acc: 26.60%
  **** New best model saved! (Acc: 26.60%)
Epoch [6/25] | Train Loss: 1.7764 | Val Loss: 1.7693 | Val Acc: 26.94%
  **** New best model saved! (Acc: 26.94%)
Epoch [7/25] | Train Loss: 1.7737 | Val Loss: 1.7668 | Val Acc: 29.63%
  **** New best model saved! (Acc: 29.63%)
Epoch [8/25] | Train Loss: 1.7682 | Val Loss: 1.7583 | Val Acc: 31.65%
  **** New best model saved! (Acc: 31.65%)
Epoch [9/25] | Train Loss: 1.7599 | Val Loss: 1.7591 | Val Acc: 32.66%
  **** New best model saved! (Acc: 32.66%)
Epoch [10/25] | T